# Creating the Testing Enviroment

For the testing environment, I have individually created 5 different user music listening history and I will begin my visualizing the embedding space features of the songs for each user.

In [4]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

#read cid from file
with open("ids/cid.txt", "r") as file:
    cid = file.read()

#read secret from file
with open("ids/secret.txt", "r") as file:
    secret = file.read()

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Collecting features from 5 pre-defined playlists that exhibit different types of music, and putting them into an embdding space for visualization.

Playslists: 
- Programming
- Jazz
- Samba
- Happy Rap
- Modern Soul

More data sourcing :]

https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge

In [3]:
session_1_uri = "spotify:playlist:4BbncuGqRY0GzR0t57PCKy"
session_2_uri = "spotify:playlist:1UMi6pQ4hqyDMbEhbshD9d"
session_3_uri = "spotify:playlist:55yqh8VajCF49DDohXO0nS"
session_4_uri = "spotify:playlist:28wpyiXjRFe8RoE65UHtWo"
session_5_uri = "spotify:playlist:1Bjl9TNi4gdi093qQzalmC"

sessions = [session_1_uri, session_2_uri, session_3_uri, session_4_uri, session_5_uri]

In [4]:
running = "37i9dQZF1EIdThJL1CyWNZ"
relaxing = "37i9dQZF1DXebxttQCq0zA"
focusing = "37i9dQZF1EIeSTCg9cy53h"
sleeping = "37i9dQZF1DWStLt4f1zJ6I"
happy = "37i9dQZF1DX3rxVfibe1L0"
sad = "37i9dQZF1EIg6gLNLe52Bd"

sessions = [running, relaxing, focusing, sleeping, happy, sad]
sessions_dict = ['running', "relaxing", "focusing", "sleeping", "happy", "sad"]

In [16]:
import json
import os
from tqdm import tqdm
import re

def write_songs_to_csv(directory = 'spotify_million_playlist_dataset/data/', write_directory = '',num_files = 1):
    file_names = os.listdir(directory)
    # opening the number of files specified in the range
    for i in range(0, num_files):
        with open(directory + file_names[i], "r") as file:

            #load the json file
            data = json.load(file)

            #iterate through the playlists
            for playlist in tqdm(data["playlists"], desc="Loading playlists"):

                playlist_data = []

                #retrieve the playlist name
                session = playlist["name"]

                #iterate through the tracks in the playlist
                for track in playlist["tracks"]:

                    #retrieve the track and artist data if possible
                    try:
                        track_obj = sp.track(track["track_uri"])
                    except:
                        continue

                    artist_obj = sp.artist(track["artist_uri"])

                    #creating the track audio features dictionary
                    track_data = sp.audio_features(track["track_uri"])[0]

                    #retrieving the artist and track names
                    artist_name = track["artist_name"]
                    track_name = track["track_name"]

                    #retrieving the artist and track popularity
                    artist_popularity = artist_obj['popularity']
                    track_popularity = track_obj["popularity"]

                    #retrieving the artist genres
                    artist_genres = artist_obj["genres"]

                    #create dictionary of track metadata
                    track_metadata = {
                        "artist_name": artist_name,
                        "track_name": track_name,
                        "artist_popularity": artist_popularity,
                        "track_popularity": track_popularity,
                        "artist_genres": artist_genres,
                        "playlist_name": session
                    }

                    #add track metadata to track features fo track data
                    track_data.update(track_metadata)
                    #add track data to playlist data list
                    playlist_data.append(track_data)

                #convert playlist list into a dataframe
                song_df = pd.DataFrame(playlist_data)
                #drop unnecessary columns
                song_df.drop(["analysis_url", "track_href", "type", 'uri'], axis=1, inplace=True)

                #get the playlist id
                pid = re.search('\d+', file_names[i])

                #write the dataframe to a csv file
                if int(playlist["pid"]) == int(pid.group()):
                    song_df.to_csv(write_directory + file_names[i] + '.csv', mode='w', header=True, index=False)
                else:
                    song_df.to_csv(write_directory + file_names[i] + '.csv', mode='a', header=False, index=False)

In [12]:
os.makedirs('playlist_dataframes', exist_ok=True) 

In [17]:
# assign directory
write_songs_to_csv(write_directory='playlist_dataframes/', num_files=1, directory='spotify_million_playlist_dataset/data/')

Loading playlists:   1%|          | 11/1000 [02:57<4:29:05, 16.33s/it]

In [9]:
songs = pd.read_csv('songs.csv', index_col=0)
songs

,Unnamed: 0.1,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,...,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artist_pop,genres,track_pop
0,0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69
1,7734,73,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,w o r k o u t,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69
2,14037,14,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,party playlist,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69
3,21536,42,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Dance mix,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69
4,24404,1,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,spin,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67494,67496,37,Jon D,3uCHI1gfOUL5j5swEh0TcH,spotify:artist:5HCypjplgh5uQezvBpOfXN,I Don't Know,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,189183,Roots,thinking of you,...,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4,47,unknown,27
67495,67499,40,Big Words,0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you,...,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4,39,australian_r&b,37
67496,67500,41,Allan Rayman,2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you,...,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4,55,canadian_contemporary_r&b modern_alternative_rock,49
67497,67501,42,Jon Jason,4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you,...,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-ana

In [19]:
#find number of unique track_uri's
songs[['track_name', 'track_uri']].value_counts()

track_name                                                                           track_uri             
One Dance                                                                            1xznGGDReH1oQq0xzbwXa3    55
HUMBLE.                                                                              7KXjTSCq5nL1LoYtL7XAwS    52
Broccoli (feat. Lil Yachty)                                                          7yyRTcZmCiyzzJlNzGC9Ol    50
Closer                                                                               7BKLCZ1jbUBVqRi2FVlTVw    46
Congratulations                                                                      3a1lNhkSLSkpJE4MSHpDu9    44
                                                                                                               ..
I Drove All Night                                                                    3zN4OLeBAN7LwAr442bB8E     1
I Dreamed a Dream                                                                    3iVUQfMt1

In [12]:
new_songs = pd.read_csv('new_songs.csv')
new_songs

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,74,dance_pop hip_hop hip_pop pop pop_rap r&b rap ...,69,"['dance_pop', 'hip_hop', 'hip_pop', 'pop', 'po...",low,Neutral
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,84,dance_pop pop post-teen_pop,83,"['dance_pop', 'pop', 'post-teen_pop']",low,Neutral
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,dance_pop pop r&b,25,"['dance_pop', 'pop', 'r&b']",high,Negative
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,82,dance_pop pop,79,"['dance_pop', 'pop']",low,Neutral
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,75,pop_rap reggae_fusion,2,"['pop_rap', 'reggae_fusion']",low,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34242,Jon D,3uCHI1gfOUL5j5swEh0TcH,I Don't Know,0.669,0.228,2,-12.119,1,0.0690,0.79200,0.065000,0.0944,0.402,83.024,47,unknown,27,['unknown'],low,Neutral
34243,Big Words,0P1oO2gREMYUCoOkzYAyFu,The Answer,0.493,0.727,1,-5.031,1,0.2170,0.08730,0.000000,0.1290,0.289,73.259,39,australian_r&b,37,['australian_r&b'],low,Neutral
34244,Allan Rayman,2oM4BuruDnEvk59IvIXCwn,25.22,0.702,0.524,7,-10.710,1,0.0793,0.33200,0.055300,0.2980,0.265,140.089,55,canadian_contemporary_r&b modern_alternative_rock,49,"['canadian_contemporary_r&b', 'modern_alternat...",low,Neutral
34245,Jon Jason,4Ri5TTUgjM96tbQZd5Ua7V,Good Feeling,0.509,0.286,8,-14.722,1,0.1230,0.40200,0.000012,0.1310,0.259,121.633,4,unknown,16,['unknown'],high,Positive


In [14]:
len(new_songs['id'].unique())

34247

In [6]:
song = sp.track('spotify:track:2vCtiBvJJZfz773yTfAxPP')
song

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7f5Zgnp2spUuuzKplmRkt7'},
    'href': 'https://api.spotify.com/v1/artists/7f5Zgnp2spUuuzKplmRkt7',
    'id': '7f5Zgnp2spUuuzKplmRkt7',
    'name': 'Lost Frequencies',
    'type': 'artist',
    'uri': 'spotify:artist:7f5Zgnp2spUuuzKplmRkt7'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/4N1e8k2o2NN932Y8xgZV1p'},
  'href': 'https://api.spotify.com/v1/albums/4N1e8k2o2NN932Y8xgZV1p',
  'id': '4N1e8k2o2NN932Y8xgZV1p',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2730094e11e9126c0c05a09bb42',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e020094e11e9126c0c05a09bb42',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048510094e11e9126c0c05a09bb42',
    'width': 64}],
  'name': 'What Is Love 2016 (Remixes)',
  'release_date': '2016-

In [5]:
# create a pandas dataframe with the features of the songs in all the playlists
def get_playlist_data(playlist_URI):

    playlist_data_list = []
    playlist = sp.playlist(playlist_URI)
    for track in playlist["tracks"]["items"]:

        #creating the track and artist objects
        track_uri = track["track"]["uri"]
        track = sp.track(track_uri)
        artist = sp.artist(track['artists'][0]['id'])

        #creating the track audio features dictionary
        track_data = sp.audio_features(track_uri)[0]

        #retrieving the artist and track names
        artist_name = track["artists"][0]["name"]
        track_name = track["name"]

        #retrieving the artist and track popularity
        artist_popularity = artist['popularity']
        track_popularity = track["popularity"]

        #retrieving the artist genres
        artist_genres = artist["genres"]

        #create dictionary of track metadata
        track_metadata = {
            "artist_name": artist_name,
            "track_name": track_name,
            "artist_popularity": artist_popularity,
            "track_popularity": track_popularity,
            "artist_genres": artist_genres
        }

        #add track metadata to track features fo track data
        track_data.update(track_metadata)

        #add track data to playlist data list
        playlist_data_list.append(track_data)

    #convert playlist list into a dataframe
    playlist_features_df = pd.DataFrame(playlist_data_list)

    #drop unnecessary columns
    playlist_features_df.drop(["analysis_url", "track_href", "type", "uri", "id"], axis=1, inplace=True)

    return playlist_features_df


In [9]:
#creating a library of playlists
playlist_list = []
for i, playlist_URI in enumerate(sessions):
    features = get_playlist_data(playlist_URI)
    features["session"] = sessions_dict[i]
    playlist_list.append(features)
playlist_df = pd.concat(playlist_list)

# playlist_df = pd.concat([playlist_df, pd.get_dummies(playlist_df["session"], prefix="session")], axis=1)
# playlist_df.drop(["session"], axis=1, inplace=True)

KeyboardInterrupt: 

In [15]:
playlist_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,artist_popularity,track_popularity,artist_genres,session
0,0.629,0.824,9,-6.530,1,0.1590,0.15500,0.000628,0.0651,0.8520,128.010,247960,4,Britney Spears,Work Bitch,83,67,"[dance pop, pop]",running
1,0.319,0.846,6,-6.109,1,0.0796,0.01030,0.000041,0.4750,0.0784,139.163,213019,4,Naughty Boy,Runnin' (Lose It All),63,64,"[dance pop, uk contemporary r&b]",running
2,0.815,0.438,0,-14.522,0,0.0346,0.21600,0.000466,0.0787,0.5520,108.965,243773,4,Survivor,Eye of the Tiger,69,72,"[album rock, hard rock, rock, soft rock]",running
3,0.506,0.959,5,-5.264,0,0.1020,0.08400,0.000655,0.1350,0.3400,133.996,213267,4,Woodkid,Run Boy Run,63,71,[chamber pop],running
4,0.686,0.735,2,-4.616,1,0.2640,0.00921,0.000660,0.3420,0.0596,171.355,322227,4,Eminem,"Lose Yourself - From ""8 Mile"" Soundtrack",93,77,"[detroit hip hop, hip hop, rap]",running
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.445,0.179,1,-12.938,1,0.0396,0.95200,0.000560,0.1070,0.1190,72.246,193080,4,Tom Odell,Heal,80,71,"[chill pop, pop]",sad
46,0.685,0.230,0,-12.525,0,0.0388,0.36700,0.000971,0.1010,0.0417,96.085,165000,4,Snøw,You Don't Want Me Anymore,68,61,"[sad lo-fi, sad rap]",sad
47,0.681,0.255,7,-16.305,1,0.0479,0.74200,0.165000,0.1050,0.2740,119.952,130947,4,Kina,Baby You're Worth It,70,67,[sad lo-fi],sad
48,0.264,0.122,9,-15.399,1,0.0349,0.88500,0.349000,0.0940,0.0735,148.658,371320,4,The Cinematic Orchestra,To Build A Home,65,72,"[downtempo, electronica, nu jazz]",sad


I am using UMAP dimensionality reduction to visualize the songs and identify clusters.

In [7]:
import umap
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [8]:
def plot_2d(component1, component2):
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=y, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        )
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=1800,height=1000)                 
    fig.layout.template = 'plotly_dark'
    
    fig.show()


def plot_3d(component1,component2,component3):
    fig = go.Figure(data=[go.Scatter3d(
            x=component1,
            y=component2,
            z=component3,
            mode='markers',
            marker=dict(
                size=10,
                color=y,                # set color to an array/list of desired values
                colorscale='Rainbow',   # choose a colorscale
                opacity=1,
                line_width=1
            )
        )])
    # tight layout
    fig.update_layout(margin=dict(l=50,r=50,b=50,t=50),width=1800,height=1000)
    fig.layout.template = 'plotly_dark'
    
    fig.show()

In [9]:
y = playlist_df["session"]
X = playlist_df.drop(["session"], axis=1)

from sklearn.preprocessing import StandardScaler
## Standardizing the data
X = StandardScaler().fit_transform(X)

reducer = umap.UMAP(random_state=42,n_neighbors=25 , min_dist=0.1, n_components=2)
embedding = reducer.fit_transform(X)

ValueError: could not convert string to float: 'Britney Spears'

In [ ]:
plot_2d(reducer.embedding_[:, 0],reducer.embedding_[:, 1])

ValueError: 
    Invalid element(s) received for the 'color' property of scatter.marker
        Invalid elements include: ['running', 'running', 'running', 'running', 'running', 'running', 'running', 'running', 'running', 'running']

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darksla

In [ ]:
y = playlist_df["session"]
X = playlist_df.drop(["session"], axis=1)

from sklearn.preprocessing import StandardScaler
## Standardizing the data
X = StandardScaler().fit_transform(X)

reducer = umap.UMAP(random_state=42,n_neighbors=15 , min_dist=.5, n_components=3)
embedding = reducer.fit_transform(X)

In [ ]:
plot_3d(reducer.embedding_[:, 0],reducer.embedding_[:, 1],reducer.embedding_[:, 2])